In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "4.Pierinae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")) and not(file.endswith("_old_backup.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
#     print(coordinate_info_dictionary)
#     print(coordinate_detail_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Anthocharis_cardamines
Exon_1 Match
[['Pieris_brassicae', ['4385399', '4385547']], ['Bicyclus_anynana', ['4385399', '4385547']], ['Maniola_jurtina', ['4385399', '4385547']]]
Exon_2 Match
[['Pieris_brassicae', ['4381399', '4381547']], ['Bicyclus_anynana', ['4381399', '4381547']], ['Maniola_jurtina', ['4381399', '4381547']]]
Exon_3 Match
[['Pieris_brassicae', ['4378979', '4379160']], ['Bicyclus_anynana', ['4378979', '4379160']], ['Maniola_jurtina', ['4378979', '4379160']]]
Exon_4 Match
[['Pieris_brassicae', ['4378750', '4378867']], ['Bicyclus_anynana', ['4378750', '4378867']], ['Maniola_jurtina', ['4378750', '4378867']]]
Exon_5 Match
[['Pieris_brassicae', ['4375366', '4375461']], ['Bicyclus_anynana', ['4375366', '4375461']], ['Maniola_jurtina', ['4375366', '4375461']]]
Exon_6 mismatch
[['Pieris_brassicae', ['4375165', '4375222']], ['Bicyclus_anynana', ['4378809', '4378890']], ['Maniola_jurtina', ['4378809', '4378890']]]
Proceed?y
Pieris_brassicae Exon_6 81_frame2 0.34 
SNLKQVEGDDLYQMDPDS

Proceed?y
Pieris_brassicae Exon_6 54_frame2 1.003 
KNIKQVEGEQMDPDS
Bicyclus_anynana Exon_6 1_frame0 1.528 
LAFKDDRPNLYSL
Maniola_jurtina Exon_6 1_frame0 1.481 

Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica -----SNMKQVDGDEMYQIDPDS-
set54_frame2    ISFSGKNIKQVEGEQM---DPDSR
                     .*:***:*::*   **** 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan TDIEQDENCDLTFED--------------PHDS--------------
set1_frame0     NDVVNTTIFDLAFKDDRPNLYSLLQNPGNPADPNQAAGCKQRFYITI
                .*: :    **:*:*              * *.              

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina TNTEQNEACDLSFQQDQES----------------------------
set1_frame0     NDVVNTTIFDLAFKDDRPNLYSLLQNPGNPADPNQAAGCKQRFYITI
                .:. :.   **:*::*: .                            

Choose Species 1 - 31
Exon_7 Match
[['Pieris_brassicae', ['2959322', '2959560']], ['Bicyclus_anynana', ['2959322', '2959560']], ['Maniola_jurtina', ['2

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina ----TNTEQNEACDLSFQQDQES-
set28_frame0    NFSGANMKQVEGDDM-YQTDSDSR
                    :* :* *. *: :* *.:* 

Choose Species 1 - 31
Exon_7 Match
[['Pieris_brassicae', ['6033296', '6033534']], ['Bicyclus_anynana', ['6033296', '6033534']], ['Maniola_jurtina', ['6033296', '6033534']]]
Exon_8 Match
[['Pieris_brassicae', ['6033923', '6034070']], ['Bicyclus_anynana', ['6033923', '6034070']], ['Maniola_jurtina', ['6033923', '6034070']]]
Exon_9 Match
[['Pieris_brassicae', ['6034347', '6034527']], ['Bicyclus_anynana', ['6034347', '6034527']], ['Maniola_jurtina', ['6034347', '6034527']]]
Exon_10 Match
[['Pieris_brassicae', ['6035277', '6035456']], ['Bicyclus_anynana', ['6035277', '6035456']], ['Maniola_jurtina', ['6035277', '6035456']]]
Exon_11 mismatch
[['Pieris_brassicae', ['6035660', '6035899']], ['Bicyclus_anynana', ['6035750', '6035899']], ['Maniola_jurtina', ['6035738', '6035899']]]
Proceed?y
Pieris_brassicae Exon_11 33_fra

Proceed?y
Pieris_brassicae Exon_11 62_frame0 0.0 
SKEAPSHMEESRPMSEGFDSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVPGVLPILPSLPVIVSDQTQIQ
Bicyclus_anynana Exon_11 62_frame0 0.825 
SKEAPSHMEESRPMSEGFDSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVPGVLPILPSLPVIVSDQTQIQ
Maniola_jurtina Exon_11 62_frame0 0.739 
QHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVPGVLPILPSLPVIVSDQTQIQ
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica SKEAPSHMEESRPMSEGFDSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVP
set62_frame0    SKEAPSHMEESRPMSEGFDSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVP
                ************************************************************

Pieris_brassica GVLPILPSLPVIVSDQTQIQ-----
set62_frame0    GVLPILPSLPVIVSDQTQIQVGFYL
                ********************     

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan T-------------------------QNVH---EATLVPQHNIGAQYLEPVPYVGPVSMP
set62_frame0    SKEAPSHMEESRPMSEGFDSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAPYVGTVSVP
              

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina ------------------------------------TQRVQEPSLVPQHGIGAQYLEPVP
set50_frame0    INIYIQFQSKEAPSHMEESRIMSEEFNSSRARMFQQQHKSVEPTVAPQHGIGAQYLEPAP
                                                     ::  **::.************.*

Maniola_jurtina YVGAVGMPGVLPLPLPPLPVIVSPDQAQLQ-----
set50_frame0    YVGTVSVPGVIPI-LPSLPVIVS-DQTQIQVGFYL
                ***:*.:***:*: **.****** **:*:*     

Choose Species 1 - 31
Exon_12 mismatch
[['Pieris_brassicae', ['6233304', '6233438']], ['Bicyclus_anynana', ['6233304', '6233426']], ['Maniola_jurtina', ['6233304', '6233426']]]
Proceed?y
Pieris_brassicae Exon_12 40_frame0 0.0 
WPWQGQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Bicyclus_anynana Exon_12 40_frame0 0.499 
GQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Maniola_jurtina Exon_12 40_frame0 0.205 
GQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica WPWQGQLQRKHEELQQMIVRQQEELRQVKE

Choose Species 1 - 31
Exon_12 mismatch
[['Pieris_brassicae', ['562113', '562247']], ['Bicyclus_anynana', ['562125', '562247']], ['Maniola_jurtina', ['562125', '562247']]]
Proceed?y
Pieris_brassicae Exon_12 48_frame0 0.097 
LPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Bicyclus_anynana Exon_12 48_frame0 0.446 
EQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Maniola_jurtina Exon_12 48_frame0 0.15 
EQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica WPWQGQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
set48_frame0    LPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
                 *** ****************************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan ----EQLQRTHRELQHMIFRQQEELRLVKEQLMLARLGIFQPMIS
set48_frame0    LPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
                    *****.*.***:**.******* *****:*****::**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurti

Exon_1 Match
[['Pieris_brassicae', ['5613808', '5613845']], ['Bicyclus_anynana', ['5613808', '5613845']], ['Maniola_jurtina', ['5613808', '5613845']]]
Exon_2 Match
[['Pieris_brassicae', ['5613044', '5613192']], ['Bicyclus_anynana', ['5613044', '5613192']], ['Maniola_jurtina', ['5613044', '5613192']]]
Exon_3 Match
[['Pieris_brassicae', ['5612275', '5612456']], ['Bicyclus_anynana', ['5612275', '5612456']], ['Maniola_jurtina', ['5612275', '5612456']]]
Exon_4 Match
[['Pieris_brassicae', ['5612073', '5612190']], ['Bicyclus_anynana', ['5612073', '5612190']], ['Maniola_jurtina', ['5612073', '5612190']]]
Exon_5 Match
[['Pieris_brassicae', ['5610961', '5611056']], ['Bicyclus_anynana', ['5610961', '5611056']], ['Maniola_jurtina', ['5610961', '5611056']]]
Exon_6 mismatch
[['Pieris_brassicae', ['5609856', '5609913']], ['Bicyclus_anynana', ['5612111', '5612162']], ['Maniola_jurtina', ['5609856', '5609913']]]
Proceed?y
Pieris_brassicae Exon_6 67_frame2 0.093 
SNMKQVDGDEMYQTDPDS
Bicyclus_anynana Exon